<a href="https://colab.research.google.com/github/lukinhasxxx/UrbanIA/blob/main/arvore_de_decisao_modelo_classificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import graphviz
import pandas as pd
from google.colab import files
!apt-get install graphviz
!pip install graphviz


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [8]:
arquivo=files.upload()

Saving brasilrev2022.CSV to brasilrev2022 (1).CSV


In [26]:
#Este é um modelo classificador, mais compatível com a proposta
cidade=pd.read_csv("brasilrev2022.CSV")

cidade.drop(index=cidade[cidade['IDHM'] ==0 ].index,inplace=True)
minimo = cidade['IDHM'].min()
maximo = cidade['IDHM'].max()
#testar se o drop das colunas deu certo e não retorna valor '0' no minimo

print(f"O menor valor de IDHM é {minimo} e o maior é {maximo}\n")

# Função para padronizar o IDHM
def padronizar_idh(valorAtual, minimo, maximo):
    return (valorAtual - minimo) / (maximo - minimo)

# agora aqui vamos aplicar a padronizaçao
cidade['IDHM_Padronizado'] = cidade['IDHM'].apply(lambda x: padronizar_idh(x, minimo, maximo))

# aqui vamos ''separar'' as classes de baixo medio e alto
cidade['IDHM_Classes'] = cidade['IDHM_Padronizado'].apply(
    lambda x: 'baixo' if x < 0.315 else 'alto' if x > 0.819 else 'médio'
)

# Mapear as classes para valores numéricos
classes = {
    'baixo': 0,
    'médio': 1,
    'alto': 2
}
cidade['IDHM_Classes'] = cidade['IDHM_Classes'].map(classes)

# aqui eh para preparar e prever os dados alvo
X = cidade[['GDP_CAPITA', 'IDHM_Educacao', 'COMP_Q', 'COMP_O', 'COMP_E', 'IDHM_Longevidade']].values
y = cidade['IDHM_Classes'].values

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Criar e treinar o modelo de classificação
modelo = DecisionTreeClassifier(max_depth=3)
modelo.fit(X_train, y_train)

# Fazer previsões
y_pred = modelo.predict(X_test)

# Avaliar o modelo, testar a acurácia
acc = accuracy_score(y_test, y_pred)
print(f'Acurácia: {acc:.4f}\n')

conf_matrix = confusion_matrix(y_test, y_pred)
print('Matriz de Confusão:\n', conf_matrix)

# Relatório de classificação
print("Relatório sobre a qualidade:\n")
print(classification_report(y_test, y_pred, target_names=['Baixo', 'Médio', 'Alto']))

# Visualizar a árvore de decisão
dot_data = export_graphviz(
    modelo,
    out_file=None,
    feature_names=['GDP_CAPITA', 'IDHM_Educacao', 'COMP_Q', 'COMP_O', 'COMP_E', 'IDHM_Longevidade'],
    class_names=['Baixo', 'Médio', 'Alto'],
    filled=True,
    rounded=True,
    special_characters=True
)

graph = graphviz.Source(dot_data)
#graph.render("cidade")  # Salva o gráfico como um arquivo .pdf ou .png dependendo do formato especificado
#executar 'graph' caso queira visualizar a árvore de decisão


O menor valor de IDHM é 0.418 e o maior é 0.862

Acurácia: 0.9539

Matriz de Confusão:
 [[  99   36    0]
 [  18 1471   10]
 [   0   13   24]]
Relatório sobre a qualidade:

              precision    recall  f1-score   support

       Baixo       0.85      0.73      0.79       135
       Médio       0.97      0.98      0.97      1499
        Alto       0.71      0.65      0.68        37

    accuracy                           0.95      1671
   macro avg       0.84      0.79      0.81      1671
weighted avg       0.95      0.95      0.95      1671

